# Advanced retrieval for RAG

Jerry Liu, the creator of LlamaIndex, recently posted this image:

<img src="images/from-simple-to-advanced-rag.jpeg" width="800">

We will cover fine-tuning and Agentic Behavior later this month;
today we'll look at 5 examples of topics under "Table Stakes" and "Advanced Retrieval".
- Metadata filters
- Hybrid search
- Reranking
- Recursive retrieval
- Small-to-big retrieval

All of these topics (and more) are covered in detail under the "Advanced Topics" tab in the LlamaIndex documentation: https://docs.llamaindex.ai/en/stable/optimizing/production_rag/ 

It's worth reading through this documentation later when you're more familiar with the basics.

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

## Load and split the document into nodes (chunks)

In [2]:
import logging
import os
import sys

from llama_index.core import Document
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding

In [3]:
# configure
filename = 'sleeping_gods.md'

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
with open(f'data/{filename}', 'r', encoding='utf-8') as file:
    document = Document(
        text = file.read(),
        metadata = {"filename": filename},
    )
print(len(document.text))

77034


In [5]:
splitter = MarkdownNodeParser()
nodes = splitter.get_nodes_from_documents([document], show_progress=False)
print(len(nodes))

97


In [6]:
for ix, node in enumerate(nodes[0:5]):
    print(f">>>{ix} {node.id_}")
    print("Metadata", node.metadata)
    print("Text", node.text[:200])
    print("\n\n")

>>>0 737d332f-22d5-4f3f-906c-d55d88f96263
Metadata {'Header_2': 'Overview (page 1)', 'filename': 'sleeping_gods.md'}
Text Overview (page 1)

1-4 players, ages 13+, 1-20 hours

"This is the Wandering Sea. The gods have brought you here, and you must wake them if you wish to return home."

In Sleeping Gods, you and up to t



>>>1 1ef2d784-342b-42f2-a9c7-43c23739bcea
Metadata {'Header_2': 'Setup (page 4)', 'filename': 'sleeping_gods.md'}
Text Setup (page 4)

Follow these instructions if you are starting a new campaign. If this is your first campaign, we recommend using the quick start guide first. If you are setting up the game to continue



>>>2 a231c621-6f71-4889-9cf0-f59e3a2265a2
Metadata {'Header_2': 'Basics (page 6)', 'filename': 'sleeping_gods.md'}
Text Basics (page 6)

Pgs. 6-9 introduce some basics of Sleeping Gods to help you get your sea legs. Turn structure and actions are explained starting on pg. 10.



>>>3 37765f08-ce45-4ca6-a541-98bd0c7c5bd5
Metadata {'Header_2': 'Basics

## Metadata filters

Let's add metadata filters so only certain nodes are retrieved.

This example is adapted from https://docs.llamaindex.ai/en/stable/examples/vector_stores/chroma_auto_retriever/

In [7]:
import re

import chromadb

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo
from llama_index.vector_stores.chroma import ChromaVectorStore

In [8]:
set(node.metadata['Header_2'] for node in nodes)

{'Basics (page 6)',
 'Challenges (page 19)',
 'Combat (page 21)',
 'Icons (page 40)',
 'Other Rules (page 28)',
 'Overview (page 1)',
 'Player Turn (page 10)',
 'Setup (page 4)',
 'Spending Command (page 17)',
 'Turn Overview (page 10)'}

In [9]:
for node in nodes:
    if 'Header_2' in node.metadata:
        node.metadata['Header_2'] = re.sub(r'\(page \d+\)$', '', node.metadata['Header_2']).strip()
set(node.metadata['Header_2'] for node in nodes)

{'Basics',
 'Challenges',
 'Combat',
 'Icons',
 'Other Rules',
 'Overview',
 'Player Turn',
 'Setup',
 'Spending Command',
 'Turn Overview'}

In [10]:
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("test")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(nodes, storage_context=storage_context)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [11]:
vector_store_info = VectorStoreInfo(
    content_info="How to play the board game Sleeping Gods",
    metadata_info=[
        MetadataInfo(
            name="Header_2",
            type="str",
            description=(
                "Board game manual topic, one of [Basics, Challenges, Combat, Icons, Other Rules, Overview, Player Turn, Setup, Spending Command, Turn Overview]"
            ),
        ),
    ],
)
retriever = VectorIndexAutoRetriever(
    index, vector_store_info=vector_store_info
)

In [12]:
# Note that the filters look good, but no nodes were retrieved!
# I repeated this step with many different queries and often got good filters but no nodes returned.
# I'm guessing this is because the chunks are way too large to match the query string: actions during turn

retriever.retrieve("What actions can I take during my turn?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: actions during my turn
Using query str: actions during my turn
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('Header_2', '==', 'Player Turn'), ('Header_2', '==', 'Turn Overview'), ('Header_2', '==', 'Spending Command')]
Using filters: [('Header_2', '==', 'Player Turn'), ('Header_2', '==', 'Turn Overview'), ('Header_2', '==', 'Spending Command')]
INFO:llama_index.core.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[]

## Hybrid search example

Hybrid search means to index each node twice: once with a dense embedding usual, and also with a "sparse" embedding. 
Hybrid search is especially important when queries contain specific keywords instead of just context. 
For example, we don't want to get results for Joseph Smith or Martin Harris for a query about Oliver Cowdery, even if their embeddings turn out to be similar.

There are several popular sparse embeddings:
- BM25 - embedding contains a column for every possible word in the index, with the value being the number of times the word appears in the node
- Splade - you can think of this as being similar to BM25, but nodes also have non-zero numbers for synonyms of the words they contain
- BGE-M3 - this is a new embedding model that supports both dense and sparse embeddings: https://github.com/FlagOpen/FlagEmbedding

We will use the Milvus index, hosted at Zilliz, which uses BGE-M3. 
To use Zilliz, you need to create a free account at https://zilliz.com/ start a cluster, 
and copy the cluster URI and TOKEN into your .env file as MILVUS_URI and MILVUS_TOKEN respectively.

You can see more hybrid search examples here:
- https://docs.llamaindex.ai/en/stable/examples/vector_stores/PineconeIndexDemo-Hybrid/
- https://docs.llamaindex.ai/en/stable/examples/vector_stores/WeaviateIndexDemo-Hybrid/
- https://docs.llamaindex.ai/en/stable/examples/vector_stores/MilvusHybridIndexDemo/

In [13]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.vector_stores.milvus import MilvusVectorStore
from pymilvus import MilvusClient

In [14]:
embed_model = OpenAIEmbedding(model= 'text-embedding-3-small')

milvus_k = 60
# delete collection if it exists
client = MilvusClient(uri=os.environ['MILVUS_URI'], token=os.environ['MILVUS_TOKEN'])
if client.has_collection('test'):
    client.drop_collection(collection_name='test')
client.close()
# hybrid uses BGE-M3 for sparse vectors
vector_store = MilvusVectorStore(
    uri=os.environ['MILVUS_URI'], 
    token=os.environ['MILVUS_TOKEN'],
    collection_name='test',
    dim=len(embed_model.get_text_embedding('foo')),
    overwrite=True,
    enable_sparse=True,  # NOTE!!!
    hybrid_ranker='RRFRanker',
    hybrid_ranker_params={'k': milvus_k},
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 4a7589f73ec949a794bf9033970dfbbd
Created new connection using: 4a7589f73ec949a794bf9033970dfbbd
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 2fb00a62120f46aeb70e6ff1152d0591
Created new connection using: 2fb00a62120f46aeb70e6ff1152d0591
Sparse embedding function is not provided, using default.


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

INFO:FlagEmbedding.BGE_M3.modeling:loading existing colbert_linear and sparse_linear---------
loading existing colbert_linear and sparse_linear---------


In [15]:
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

In [16]:
# add the nodes to the index
index.insert_nodes(nodes)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


/Users/dallan/dallanq/PathwayInternshipPresentations/.venv/lib/python3.11/site-packages/transformers/models/xlm_roberta/modeling_xlm_roberta.py:1578: UserWarning: cumsum_out_mps supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UnaryOps.mm:425.)
  incremental_indices = (torch.cumsum(mask, dim=1).type_as(mask) + past_key_values_length) * mask


In [17]:
# create a retriever from the index
top_k = 3
sparse_top_k = top_k * 5
retriever = index.as_retriever(
    vector_store_query_mode=VectorStoreQueryMode.HYBRID,  # NOTE!!!
    similarity_top_k=top_k,
    sparse_top_k=sparse_top_k,
)

In [18]:
query = 'What is fatigue?'
nodes = retriever.retrieve(query)
for ix, node in enumerate(nodes):
    print(f"\n\n{ix} Node={node.id_}")
    print(node.text[:1024])

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


0 Node=bb919c95-995d-41b5-9f68-11623a6a40d0
Fatigue (page 8)

When crew members participate in challenges, they gain fatigue, represented by a fatigue token. Each crew member can hold up to 2 fatigue tokens.

Each fatigue token is double-sided. If a crew member has only 1 token, place the blank side face up. If a crew member has a 2nd fatigue token, it should have the "-1 damage" side face up. This causes the crew member to deal -1 damage in combat.

A crew member with 2 fatigue tokens cannot participate in challenges (pg. 19), but can continue to participate in combat (pg. 21).

You can remove fatigue mainly by cooking recipes or performing a port action.


1 Node=ae2addc7-cd3c-4b5a-94fe-974e5fb33a4f
Failure (page 20)

If you fail, apply the consequences listed after the word "fail." These are some of the possible consequences

## Reranking example

It's really difficult to capture all of the nuances present in a sentence or paragraph of text in a sequence of numbers such that sentences/paragraphs with similar meaning end up having vectors with high cosine similarity. It's much easier to write an algorithm that takes two text sentences at once and returns how similar they are.

Therefore, one popular approach in RAG systems is to use a "re-ranker" model that takes the query and the similar nodes returned from the index. 
This approach issues (query, node) pairs to a re-ranker model to score how relevant each returned node is to the query and uses those scores to re-rank the nodes.
The top nodes are then fed to the LLM for generating the answer.

There are many re-ranker models. CoHere seems to be the most popular. Some people suggest using multiple re-ranking models and then averaging the results to get the top nodes.

Let's re-rank the nodes returned from the previous query. To run this code you will need to create a free account at https://cohere.com/ and copy the API key into your .env file as COHERE_API_KEY.

This example is adapted from https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/CohereRerank/

In [19]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

In [20]:
cohere_rerank = CohereRerank(
    api_key=os.environ["COHERE_API_KEY"], 
    top_n=2,
)

In [21]:
reranked_nodes = cohere_rerank.postprocess_nodes(
    nodes=nodes,
    query_str=query,
)

INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"


In [22]:
for ix, node in enumerate(reranked_nodes):
    print(f"\n\n{ix} Node={node.id_} Score={node.score}")
    print(node.text[:1024])



0 Node=bb919c95-995d-41b5-9f68-11623a6a40d0 Score=0.6674729
Fatigue (page 8)

When crew members participate in challenges, they gain fatigue, represented by a fatigue token. Each crew member can hold up to 2 fatigue tokens.

Each fatigue token is double-sided. If a crew member has only 1 token, place the blank side face up. If a crew member has a 2nd fatigue token, it should have the "-1 damage" side face up. This causes the crew member to deal -1 damage in combat.

A crew member with 2 fatigue tokens cannot participate in challenges (pg. 19), but can continue to participate in combat (pg. 21).

You can remove fatigue mainly by cooking recipes or performing a port action.


1 Node=ed00aa5e-eca5-4e1e-8868-1c46cc13ff5b Score=0.22371607
Challenges (page 19)

A challenge is a test of skill that you must overcome. Each challenge is associated with one of the five crew skills: strength, cunning, savvy, perception, or craft.

A basic challenge looks like this:

Strength 5 Fail: -3 Health

I

## Small-to-big retrieval example

The idea here is to create small nodes (e.g., a node for each sentence) but to pass the node with surrounding context (e.g., the entire paragraph or the previous and next 5 sentences) to the LLM to answer the question.

LlamaIndex includes a *SentenceWindowParser* that makes it easy to add previous and next sentences as metadata to a node, 
and a *MetadataReplacementNodePostProcessor* that makes it easy to add those sentences from the metadata back to the node text.
Let's try it out.

This example is adapted from https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/MetadataReplacementDemo/

In [26]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.llms.openai import OpenAI

In [27]:
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [28]:
nodes = node_parser.get_nodes_from_documents([document])
print(len(nodes))
print(nodes[2].metadata)
print(nodes[2].text)

998
{'window': '## Overview (page 1)\n\n1-4 players, ages 13+, 1-20 hours\n\n"This is the Wandering Sea.  The gods have brought you here, and you must wake them if you wish to return home."\n\n In Sleeping Gods, you and up to three friends become Captain Sofi Odessa and her crew, lost in a strange world in 1929 on your steamship, the Manticore.  You must work together to survive, exploring exotic islands, meeting new characters, and seeking out the totems of the gods so that you can return home.\n\n Sleeping Gods is a campaign game.  Each session can last as long as you want. ', 'original_text': 'In Sleeping Gods, you and up to three friends become Captain Sofi Odessa and her crew, lost in a strange world in 1929 on your steamship, the Manticore. ', 'filename': 'sleeping_gods.md'}
In Sleeping Gods, you and up to three friends become Captain Sofi Odessa and her crew, lost in a strange world in 1929 on your steamship, the Manticore. 


In [29]:
# add the nodes to an index
sentence_index = VectorStoreIndex(nodes)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.open

In [30]:
retriever = sentence_index.as_retriever(top_k=2)
node_postprocessor = MetadataReplacementPostProcessor(target_metadata_key="window")

In [31]:
query = 'What is fatigue?'
nodes = retriever.retrieve(query)

for ix, node in enumerate(nodes):
    print(f"\n\n{ix} Node={node.id_}")
    print('Metadata', node.metadata)
    print('Text', node.text)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


0 Node=9ea5d975-f66d-4336-902d-2b0ee28a6699
Metadata {'window': '#### Challenge (page 8)\n\nYou may be faced with challenges you must overcome, usually when reading from the storybook but also when you draw event cards.  It involves selecting crew members to participate and drawing fate in an effort to reach or exceed a specified challenge number (see pg.  19).\n\n #### Fatigue (page 8)\n\nWhen crew members participate in challenges, they gain fatigue, represented by a fatigue token.  Each crew member can hold up to 2 fatigue tokens.\n\n Each fatigue token is double-sided.  If a crew member has only 1 token, place the blank side face up. ', 'original_text': '#### Fatigue (page 8)\n\nWhen crew members participate in challenges, they gain fatigue, represented by a fatigue token. ', 'filename': 'sleeping_gods.md'}
Text #### Fatigu

In [32]:
processed_nodes = node_postprocessor.postprocess_nodes(nodes)

for ix, node in enumerate(processed_nodes):
    print(f"\n\n{ix} Node={node.id_}")
    print('Metadata', node.metadata)
    print('Text', node.text)



0 Node=9ea5d975-f66d-4336-902d-2b0ee28a6699
Metadata {'window': '#### Challenge (page 8)\n\nYou may be faced with challenges you must overcome, usually when reading from the storybook but also when you draw event cards.  It involves selecting crew members to participate and drawing fate in an effort to reach or exceed a specified challenge number (see pg.  19).\n\n #### Fatigue (page 8)\n\nWhen crew members participate in challenges, they gain fatigue, represented by a fatigue token.  Each crew member can hold up to 2 fatigue tokens.\n\n Each fatigue token is double-sided.  If a crew member has only 1 token, place the blank side face up. ', 'original_text': '#### Fatigue (page 8)\n\nWhen crew members participate in challenges, they gain fatigue, represented by a fatigue token. ', 'filename': 'sleeping_gods.md'}
Text #### Challenge (page 8)

You may be faced with challenges you must overcome, usually when reading from the storybook but also when you draw event cards.  It involves sele

## Recursive retrieval example

Suppose we want to index content from many documents. 
One way to search this content is to create a separate index for each document, and to create a top-level index that contains summaries of each document.
To process a query, you first query the top-level index to see which document is most-likely to contain an answer, then re-issue the query on the index for that document.
This is called *Recurisve retrieval*.

Let's index content for the board games Sleeping Gods and also Everdell.

This example is adapted from https://docs.llamaindex.ai/en/stable/examples/retrievers/auto_vs_recursive_retriever/

In [33]:
from llama_index.core import SimpleDirectoryReader, SummaryIndex
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.schema import IndexNode
from llama_index.llms.openai import OpenAI

In [34]:
llm = OpenAI("gpt-4")
callback_manager = CallbackManager([LlamaDebugHandler()])

# we'll just do dumb splitting for this example
splitter = SentenceSplitter(chunk_size=256)

titles = ['everdell', 'sleeping_gods']

In [35]:
# load documents
docs_dict = {}
for title in titles:
    with open(f'data/{title}.md', 'r', encoding='utf-8') as file:
        document = Document(
            text = file.read(),
            metadata = {"title": title},
        )
    docs_dict[title] = document
print(len(docs_dict))

2


In [36]:
# define retrievers and summary nodes
summary_nodes = []
retrievers = {}

for title in titles:
    # build vector index
    vector_index = VectorStoreIndex.from_documents(
        [docs_dict[title]],
        embed_model=embed_model,
        transformations=[splitter],
        callback_manager=callback_manager,
    )
    # define retriever
    retrievers[title] = vector_index.as_retriever(top_k=2)

    # generate a summary
    summary_index = SummaryIndex.from_documents(
        [docs_dict[title]], callback_manager=callback_manager
    )

    summarizer = summary_index.as_query_engine(
        response_mode="tree_summarize", llm=llm
    )
    response = summarizer.query(
        f"Give me a summary of {title}"
    )

    summary = response.response

    print(f"**Summary for {title}: {summary}")
    node = IndexNode(text=summary, index_id=title)
    summary_nodes.append(node)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 0.667611 seconds
**********
**********
Trace: index_construction
**********
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
**Summary for everdell: Everdell is a board game where players take on the role of critters building cities in an enchanting forest. The game begins in late winter and ends as the next winter approaches, with each player taking turns to perform actions such as placing a worker, playing a card, or preparing for a season. 

Players can visit various locations to gather resources, draw more cards, host events, or embark on a journey. There are two types of locations: exclusive and shared. The game also involves playing card

In [37]:
# define top-level retriever
top_vector_index = VectorStoreIndex(
    summary_nodes, 
    embed_model=embed_model,
    transformations=[splitter], 
    callback_manager=callback_manager,
)
top_retriever = top_vector_index.as_retriever(similarity_top_k=1)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 0.253167 seconds
**********


In [38]:
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": top_retriever, **retrievers},
    verbose=True,
)

In [39]:
# run recursive retriever
query = "What is the difference between exclusive and shared locations in the forest?"
nodes = recursive_retriever.retrieve(query)
for ix, node in enumerate(nodes):
    print(f"\n\n>>> {ix} Node={node.id_}")
    print(node.text[:1024])
    # print(node.node.get_content())

Retrieving with query id None: What is the difference between exclusive and shared locations in the forest?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Retrieved node with id, entering: everdell
Retrieving with query id everdell: What is the difference between exclusive and shared locations in the forest?
Retrieving text node: There are two types of locations: exclusive and shared (portrayed with an open encircling ring). Only 1 worker may visit an exclusive location. Multiple workers, even of the same color, may visit a shared location.

To visit a location, place one of your workers on any available symbol and immediately take the listed resources or perform the action. That worker is then considered deployed to that location until you bring them back with the Prepare for Season action.

Example: Placing a worker on the second basic action space gains the player two twig

### Let's break down how the recursive retriever works

In [40]:
# query the top retriever
nodes = top_retriever.retrieve(query)
index_id = nodes[0].node.index_id
print(index_id)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: query
    |_CBEventType.RETRIEVE -> 0.190008 seconds
      |_CBEventType.EMBEDDING -> 0.185999 seconds
**********
everdell


In [41]:
retrievers[index_id].retrieve(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: query
    |_CBEventType.RETRIEVE -> 0.170884 seconds
      |_CBEventType.EMBEDDING -> 0.156394 seconds
**********


[NodeWithScore(node=TextNode(id_='f0aadbc2-8e57-4339-82c0-9de48b8f165e', embedding=None, metadata={'title': 'everdell'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='398df769-8b34-4313-8837-30bf7ea1dc05', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': 'everdell'}, hash='1eec132a7e78d74d7c1c3ef971d658520ca36c180a4129e1c2b3c21bd2e750c7'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='0b7dbfc9-9817-46bb-98b0-41cdd79a3687', node_type=<ObjectType.TEXT: '1'>, metadata={'title': 'everdell'}, hash='ebbeaf08eb03bef2fad62843a43a1230747725a75159882dad0072e08f0bb6e3'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='e1a8fcd5-6146-4be5-af63-79f647972f7d', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='6ffd14501c6bb2e437bb765c4f09664d3aa67ed11246534ebf02a20d2da2342e')}, text='There are two types of locations: exclusive and shared (portrayed with an open encircling ring). Onl